In [1]:
from selenium import webdriver
import pandas as pd
import time

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
list = []

driver = webdriver.Chrome()
driver.get(url)

In [3]:
# Locate all the values
table = driver.find_element_by_class_name('wikitable')
td_content = table.find_elements_by_tag_name('td')

for td in td_content:
    text = td.text
    list.append(text)

driver.close()

In [4]:
list[:5]

['M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned']

In [5]:
import pandas as pd

In [6]:
# Convert list to Dataframe
df = pd.DataFrame({'Postcode': list[::3], 
                   'Borough': list[1::3], 
                   'Neighbourhood': list[2::3]})
df = df[['Postcode', 'Borough', 'Neighbourhood']]

In [7]:
# Remove cells with a borough that is not assigned
df = df[df.Borough != 'Not assigned']

In [8]:
# Identify Neighbourhood with not assigned value
df_2 = df[df.Neighbourhood == 'Not assigned']
df_2

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [9]:
# Make Neighbourhood the same as Borough
df_2.iloc[0, 2] = df_2.iloc[0, 1]

c:\users\geo\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\users\geo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
df = df[df.Neighbourhood != 'Not assigned'].append(df_2)
df[-5:]

,Postcode,Borough,Neighbourhood
284,M8Z,Etobicoke,Mimico NW
285,M8Z,Etobicoke,The Queensway West
286,M8Z,Etobicoke,Royal York South West
287,M8Z,Etobicoke,South of Bloor
8,M7A,Queen's Park,Queen's Park


In [11]:
# Combine neighborhood names under the same postalcode into one string
df_l = df[['Postcode', 'Neighbourhood']]
neigh_list = []
postcode_list = df_l.Postcode.unique()

for pcode in postcode_list:
    df_pcode = df_l[df_l.Postcode == pcode]
    text = ''
    for neigh in df_pcode.Neighbourhood:
        text = text+','+neigh
    text = text[1:]
    neigh_list.append(text)
    
len(postcode_list)

103

In [12]:
df_l = pd.DataFrame({'Postcode': postcode_list, 'Neighbourhood': neigh_list})

In [13]:
# Drop duplicated postalcodes
df_r = df[['Postcode', 'Borough']]
df_r.drop_duplicates(inplace=True)

c:\users\geo\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
# Merge two dataframes into one
new_df = pd.merge(df_l, df_r)
new_df = new_df[['Postcode', 'Borough', 'Neighbourhood']]
new_df.sort_values(by='Postcode', inplace=True)
new_df.reset_index(inplace=True)
new_df.drop('index', axis=1, inplace=True)

In [15]:
new_df.shape

(103, 3)

In [16]:
new_df.to_csv('Canada_postal_codes.csv', index=False)